In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import scipy.stats as stats
import statsmodels.api as sm
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.linear_model import Ridge,Lasso,ElasticNet
from sklearn.metrics import accuracy_score,root_mean_squared_error,r2_score

In [ ]:
data=pd.read_excel(r"C:\Users\Tippu\Downloads\concrete+compressive+strength\Concrete_Data.xls")
data.head(5)
data.tail(5)

In [ ]:
data.info() #all the concrete mix aggregates are in same unit 

#split input and output variable
y=data['Concrete compressive strength(MPa, megapascals) ']
x=data.drop(['Concrete compressive strength(MPa, megapascals) '],axis=1)

In [ ]:
data.describe()

In [ ]:
data.shape
data.columns

In [ ]:
data.duplicated().sum()  # duplicates are present around 25
data.drop_duplicates(inplace=True)
data.duplicated().sum()

In [ ]:
data.isna().sum() # no missing values are present

###UNIVARIATE ANALYSIS

In [ ]:
#histogram
fig=px.histogram(data['Age (day)'],nbins=300)
fig.show()

fig=px.histogram(data['Concrete compressive strength(MPa, megapascals) '],nbins=100)
fig.show()

# Density Plot
sns.kdeplot(data['Age (day)'])   #A smoothed version of the histogram that estimates the probability density function of the variable.
plt.title('Density Plot of Age Variable')
plt.xlabel('Value')
plt.ylabel('Density')
plt.show()

In [ ]:

#histogram
for col in data.columns:
    sns.kdeplot(data[col])   #A smoothed version of the histogram that estimates the probability density function of the variable.
    plt.title(f'Density Plot of {col} Variable')
    plt.xlabel('Value')
    plt.ylabel('Density')
    plt.show()

In [ ]:
#box plot
# Create subplots
fig, axes = plt.subplots(1,len(data.columns), figsize=(10, 6))  # 2x2 grid of plots

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Define the column names for easy reference
columns = data.columns

# Plot each column in its own subplot
for ax, column in zip(axes, columns):
    ax.boxplot(data[column])
    
    ax.set_ylabel(column)

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
fig=px.box(data['Age (day)'])
fig.show()

fig=px.box(data['Concrete compressive strength(MPa, megapascals) '])
fig.show()

In [ ]:
# the skewness is positive for the variables except coarse and fine aggregate which have negative skewness
#Positive Skewness: A distribution of income where most people earn below the average income but a few earn significantly more, pulling the mean to the right.
#Negative Skewness: A distribution of test scores where most students score above the average, but a few score very low, pulling the mean to the left.


#Excess Kurtosis = 0: Indicates a normal distribution.
# Positive Excess Kurtosis: Indicates a distribution with heavier tails and a sharper peak than the normal distribution (leptokurtic). This means more data points are in the tails or peak of the distribution.
# Negative Excess Kurtosis: Indicates a distribution with lighter tails and a flatter peak than the normal distribution (platykurtic). This means fewer data points are in the tails or peak of the distribution.

# Example:

#Positive Kurtosis: A distribution of extreme stock returns where there are more extreme values (both high and low) than in a normal distribution.
#Negative Kurtosis: A distribution of heights where almost all values are clustered around the mean with very few extreme values.

In [ ]:
for col in data.columns:
    skewness=data[col].skew()
    kurtosis=data[col].kurt()
    print(f" The {col} has \nskewness:{skewness}\n kurtosis:{kurtosis}\n")


In [ ]:
def qq_plot(data,dist='norm'):
    stats.probplot(data,dist=dist,plot=plt)
    plt.title(f'Q-Q Plot against {dist.capitalize()} Distribution ')
    plt.show()

#Interpreting the Q-Q Plot
#Points on the Line: If the points lie approximately on the reference line (usually a 45-degree line), the data is likely normally distributed.
#S-Shaped Curve: If the points form an S-shaped curve, the data may have heavier tails than a normal distribution (leptokurtic) or lighter tails (platykurtic).
#Systematic Deviations: Deviations from the line, especially if systematic, indicate that the data may not follow the theoretical distribution.

In [ ]:
qq_plot(data['Cement (component 1)(kg in a m^3 mixture)']) # qq plot 
for col in data.columns:   # closely the data falls on the line with a slight deviation 
    qq_plot(data[col])

##BIVARIATE ANALYSIS

In [ ]:
corr_data=data.corr()
corr_data
# flyash,blast furnance slag has a negative correlation which make sense as it is added to reduce cement content also to increase strength and reduce carbon footprints
# water and superplasticizer has a negative  medium corelation

sns.heatmap(corr_data, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix Heatmap')
plt.show()


In [ ]:
#scatter plot
def scatt_plot(data1,data2):
    plt.scatter(data1,data2)
    plt.title('Scatter Plot of Variable1 vs Variable2')
    plt.xlabel(f'Variable1')
    plt.ylabel(f'Variable2')
    plt.show()
    
scatt_plot(data['Cement (component 1)(kg in a m^3 mixture)'],data['Blast Furnace Slag (component 2)(kg in a m^3 mixture)'])

In [ ]:
#pair plot   #To visualize pairwise relationships between multiple variables in a grid format.
sns.pairplot(data[['Cement (component 1)(kg in a m^3 mixture)', 'Blast Furnace Slag (component 2)(kg in a m^3 mixture)','Fly Ash (component 3)(kg in a m^3 mixture)','Water  (component 4)(kg in a m^3 mixture)','Superplasticizer (component 5)(kg in a m^3 mixture)','Concrete compressive strength(MPa, megapascals) ']])  
plt.title('Pair Plot')
plt.show()


In [ ]:
k=data[data['Age (day)']==365]   #14
k_270=data[data['Age (day)']==270] #13
k1=data[(data['Age (day)'] > 180) & (data['Age (day)'] <=365) ] #33
k1.shape
k.describe()
k1.sample(5) 

In [ ]:
#scaling the data
inst1=StandardScaler()
scaled_data=pd.DataFrame(inst1.fit_transform(x),columns=x.columns)

scaled_data.describe()

#MODEL TRAINING

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(scaled_data,y,test_size=0.2,shuffle=True,random_state=42)
x_train

In [ ]:
#model-1
#linear regression
# add a const term 
x1=sm.add_constant(x_train)
model_1a=sm.OLS(y_train,x1).fit()   # model is not performing well with adjusted r2=0.607
model_1a.summary()
model_1a.summary2()

#predict the test set
x2=sm.add_constant(x_test)
predict_1a=pd.DataFrame(model_1a.predict(x2),columns=['predicted']).set_index(x_test.index)
predict_1a

root_mean_squared_error(predict_1a,y_test) #9.796
r2_score(predict_1a,y_test) #0.423

In [ ]:
error=predict_1a['predicted']-y_test
fig=px.scatter(error,predict_1a['predicted'])
fig.show()
#the plot shows a random scatter, this suggests that the model's assumptions are valid. 
# However, if you see patterns (like a funnel shape), it might indicate issues such as heteroscedasticity (non-constant variance)


fig=px.histogram(error,nbins=10)
fig.show()
#  As residuals are approximately normally distributed ,it supports the validity of the regression model's assumptions.

In [ ]:
#ridge regression(L2 NORM)
r=Ridge()
param_grid={'alpha':np.logspace(-4,4,50)}
grid_search=GridSearchCV(r,param_grid=param_grid,scoring='neg_mean_squared_error',cv=5,n_jobs=-1)
grid_search.fit(x_train,y_train)

# Best model
best_ridge = grid_search.best_estimator_
alpha_value= grid_search.best_params_['alpha']
print("Best alpha:", grid_search.best_params_['alpha'])
print("Best model coefficients:", best_ridge.coef_)

pred=best_ridge.predict(x_test)
r2_score(y_test,pred)#62.756

#NOTE:
#It helps to prevent overfitting by shrinking the regression coefficients.and a term λ∥β∥2 is aded to cost fn.
#A larger λ means more shrinkage of the coefficients towards zero.
# ridge regression shrinks the coefficients and helps mitigate issues of multicollinearity and overfitting.


In [ ]:
#lasso regression(l1 norm)
l=Lasso()
param_grid={'alpha':np.logspace(-4,4,50)}
grid_search1=GridSearchCV(l,param_grid=param_grid,scoring='neg_mean_squared_error',cv=5,n_jobs=-1)
grid_search1.fit(x_train,y_train)

# Best model
best_lasso1 = grid_search1.best_estimator_
alpha_value1= grid_search1.best_params_['alpha'] #0.0001
print("Best alpha:", grid_search1.best_params_['alpha'])
print("Best model coefficients:", best_lasso1.coef_)

pred1=best_lasso1.predict(x_test)
r2_score(y_test,pred1) #62.75

#NOTE:

# a penalty equivalent to the absolute value of the magnitude of coefficients(λ∥β∥).
# It be useful in scenarios where we need to handle multicollinearity, reduce model complexity, or select variables.
#The L1 norm tends to shrink some coefficients exactly to zero when the regularization parameter λ is sufficiently large. 
# This results in a sparse model, where only a subset of the original variables are retained, leading to variable selection.
# The addition of the L1 penalty term introduces bias into the estimates of the coefficients, which can help reduce the variance of the model.

In [ ]:
# Elastic Net Regression
e=ElasticNet()
param_grid={'alpha':np.logspace(-4,4,50),'l1_ratio':np.random.uniform(0,1,15)}
grid_search2=GridSearchCV(e,param_grid=param_grid,scoring='neg_mean_squared_error',cv=5,n_jobs=-1)
grid_search2.fit(x_train,y_train)

best_elastic_values=grid_search2.best_estimator_
print("Best model coefficients:", best_elastic_values.coef_)

pred_elastic_test=best_elastic_values.predict(x_test)
r2_score(y_test,pred_elastic_test) #62.75
pred_elastic_train=best_elastic_values.predict(x_train)


In [ ]:
#mode1-2
instance_2a=KNeighborsRegressor()
model_2a=instance_2a.fit(x_train,y_train)
predict_2a=pd.DataFrame(model_2a.predict(x_test),columns=['predicted']).set_index(x_test.index)
predict_2a

root_mean_squared_error(predict_2a,y_test) #8.65
r2_score(predict_2a,y_test) #0.552
#  KNN is a non-parametric method and does not produce parameters (like coefficients) that can be summarized statistically.

In [ ]:
#hyper tuning of decision tree
param_grid_knn={'n_neighbors':[5,10,15,20],
                'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
    }

grid_search_knn=GridSearchCV(instance_2a,param_grid=param_grid_knn,scoring='neg_root_mean_squared_error',cv=3,n_jobs=-1)
grid_search_knn.fit(x_train,y_train)

best_param_knn=grid_search_knn.best_estimator_ 

predict_knn_train=pd.DataFrame(best_param_knn.predict(x_train),columns=['predicted']).set_index(x_train.index)
predict_knn_train

#on test data
predict_knn_test=pd.DataFrame(best_param_knn.predict(x_test),columns=['predicted']).set_index(x_test.index)

r2_score(predict_knn_test,y_test) #0.66

In [ ]:
#mode1-3
instance_3a=DecisionTreeRegressor(random_state=0)
model_3a=instance_3a.fit(x_train,y_train)
predict_3a=pd.DataFrame(model_3a.predict(x_test),columns=['predicted']).set_index(x_test.index)
predict_3a


root_mean_squared_error(predict_3a,y_test) #6.92
r2_score(predict_3a,y_test) #0.7876


In [ ]:
#hyper tuning of decision tree
param_grid_dt={'criterion':['squared_error', 'friedman_mse','absolute_error'],
            'splitter':['best', 'random'],
            'max_depth':[None,5,10,15],
            'min_samples_split':[2,3,5,10,15,20],
            'min_samples_leaf':[1,2,3,5,10,15],
            'max_features':['sqrt', 'log2'],
            'random_state':[0,42,125]}

grid_search_dt=GridSearchCV(instance_3a,param_grid=param_grid_dt,scoring='neg_mean_squared_error',cv=5,n_jobs=-1)
grid_search_dt.fit(x_train,y_train)

best_param=grid_search_dt.best_estimator_

#prediction
predict_3b=pd.DataFrame(best_param.predict(x_test),columns=['predicted']).set_index(x_test.index)
predict_3b
root_mean_squared_error(predict_3b,y_test)#6.8859
r2_score(predict_3b,y_test) #0.7794

In [ ]:
'''
#calculate the cost complexity pruning path, which provides the effective alphas and the corresponding total leaf impurities for the subtrees
# formed by pruning.
#Computing Cost Complexity Pruning Path gives us a series of trees pruned at different levels of complexity, controlled by a parameter called ccp_alpha.
#ccp_alphas: List of values of the complexity parameter alpha.;As ccp_alpha increases, more of the tree is pruned, resulting in a simpler tree.
#impurities: Total impurity of the leaves of the pruned trees.

#This function calculates a series of nested subtrees by incrementally pruning the tree based on a cost complexity measure.   

'''

import matplotlib.pyplot as plt
# Get the pruning path
path = model_3a.cost_complexity_pruning_path(x_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

# Plot the total impurity vs effective alpha
plt.figure(figsize=(10, 6))
plt.plot(ccp_alphas[:-1], impurities[:-1], marker='o', drawstyle='steps-post')
plt.xlabel("Effective Alpha")
plt.ylabel("Total Impurity")
plt.title("Total Impurity vs Effective Alpha")
plt.show()


In [ ]:
#Train decision trees for each alpha value in the pruning path:
pruned_trees = []
for ccp_alpha in ccp_alphas:
    pruned_tree = DecisionTreeRegressor(ccp_alpha=ccp_alpha)
    pruned_tree.fit(x_train, y_train)
    pruned_trees.append(pruned_tree)

In [ ]:
# Evaluate the performance of each pruned tree on the validation set
train_scores = [tree.score(x_train, y_train) for tree in pruned_trees]
test_scores = [tree.score(x_test, y_test) for tree in pruned_trees]

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(ccp_alphas, train_scores, label='Train Score', marker='o')
plt.plot(ccp_alphas, test_scores, label='Test Score', marker='o')
plt.xlabel('Alpha')
plt.ylabel('R^2 Score')
plt.title('Cost Complexity Pruning')
plt.legend()
plt.show()

'''
#The score() method evaluates the performance of a ML model by calculating how well it predicts the target values based on the provided feature data.
# Specifically, in the context of a decision tree (or any supervised learning model),it typically measures accuracy. Here’s a breakdown:

#Accuracy: For classification tasks, score() returns the accuracy of the model, which is the ratio of correctly predicted observations to the total observations.
# It's calculated as: Accuracy=Number of correct predictions/ Total number of predictions

#R² Score: For regression tasks, the method may return the R² (coefficient of determination) score, which indicates how well the model's predictions 
# match the actual data. The R² score varies from 0 to 1, where 1 indicates perfec
'''

In [ ]:
# Select the pruned tree with the highest test score
best_tree_index = test_scores.index(max(test_scores))
best_tree = pruned_trees[best_tree_index]
Best_alpha=ccp_alphas[best_tree_index]
# Evaluate the best tree
print(f"Best alpha: {ccp_alphas[best_tree_index]}")
print(f"Train score: {train_scores[best_tree_index]}")
print(f"Test score: {test_scores[best_tree_index]}")
best_tree

In [ ]:
bt1=DecisionTreeRegressor(random_state=0,ccp_alpha=0.01507668645580644)
bt=bt1.fit(x_train,y_train)
#prediction
predict_3b1=pd.DataFrame(bt.predict(x_test),columns=['predicted']).set_index(x_test.index)
predict_3b1
root_mean_squared_error(predict_3b1,y_test)#6.8864
r2_score(predict_3b1,y_test) #0.7823

#########################################################
bt2=DecisionTreeRegressor(criterion='squared_error',random_state=0,ccp_alpha=0.01507668645580644,max_features='sqrt',max_depth=10,min_samples_split=5,min_samples_leaf=3)
bt2=bt2.fit(x_train,y_train)
#prediction
predict_3b2=pd.DataFrame(bt2.predict(x_test),columns=['predicted']).set_index(x_test.index)
predict_3b2
root_mean_squared_error(predict_3b2,y_test)#6.95
r2_score(predict_3b2,y_test) #0.7642

In [ ]:
#hyper tuning of decision tree
import numpy as np
param_grid_dt1={'ccp_alpha': np.linspace(0, 0.1, num=50)
}

grid_search_dt1=GridSearchCV(best_tree,param_grid=param_grid_dt1,scoring='neg_root_mean_squared_error',cv=3,n_jobs=-1)
grid_search_dt1.fit(x_train,y_train)

best_param1=grid_search_dt1.best_estimator_  #best_param1 #ccp_alpha=0.07142857142857144

#prediction
predict_dt_test=pd.DataFrame(best_param1.predict(x_test),columns=['predicted']).set_index(x_test.index)
predict_dt_test
root_mean_squared_error(predict_dt_test,y_test)#6.8991
r2_score(predict_dt_test,y_test) #0.78587

#prediction
predict_dt_train=pd.DataFrame(best_param1.predict(x_train),columns=['predicted']).set_index(x_train.index)
predict_dt_train


In [ ]:
#mode1-4
instance_4a=RandomForestRegressor(oob_score=True,verbose=True,random_state=42)
model_4a=instance_4a.fit(x_train,y_train)
predict_4a=pd.DataFrame(model_4a.predict(x_test),columns=['predicted']).set_index(x_test.index)
predict_4a

root_mean_squared_error(predict_4a,y_test) #5.624
r2_score(predict_4a,y_test) #0.8536

In [ ]:
#hyper tuning of random forest
param_grid_rdt={'ccp_alpha': np.linspace(0, 0.8, num=50)}

grid_search_rdt=GridSearchCV(instance_4a,param_grid=param_grid_rdt,scoring='neg_mean_squared_error',cv=5,n_jobs=-1)
grid_search_rdt.fit(x_train,y_train)

best_param_rdt=grid_search_rdt.best_estimator_

#prediction
predict_4b=pd.DataFrame(best_param_rdt.predict(x_test),columns=['predicted']).set_index(x_test.index)
predict_4b
root_mean_squared_error(predict_4b,y_test)#5.6
r2_score(predict_4b,y_test) #0.8536


predict_rdt_train=pd.DataFrame(best_param_rdt.predict(x_train),columns=['predicted']).set_index(x_train.index)
predict_rdt_train


In [ ]:
#model-5
#xgb boost
instance_5a=XGBRegressor()
model_5a=instance_5a.fit(x_train,y_train)
predict_5a=pd.DataFrame(model_5a.predict(x_test),columns=['predicted']).set_index(x_test.index)
predict_5a

root_mean_squared_error(predict_5a,y_test) #4.452
r2_score(predict_5a,y_test) #0.9151

predict_xg_train=pd.DataFrame(model_5a.predict(x_train),columns=['predicted']).set_index(x_train.index)
predict_xg_train

In [ ]:
#combine predictions into a new feature matrix for meta model
x_meta_train=np.column_stack((pred_elastic_train,predict_knn_train,predict_dt_train,predict_rdt_train,predict_xg_train))
x_meta_train

# Combine test predictions
x_meta_test=np.column_stack((pred_elastic_test,predict_knn_test,predict_dt_test,predict_4b,predict_5a))
x_meta_test

In [ ]:
meta_model=Ridge()
meta_model.fit(x_meta_train,y_train)

# Predict with meta-model
y_pred_ = meta_model.predict(x_meta_test)


rmse_ =root_mean_squared_error(y_pred_,y_test) 
print(f'Mean Squared Error of Stacking Regressor: {rmse_:.2f}')#7.65

In [ ]:
meta_model1=XGBRegressor()
meta_model1.fit(x_meta_train,y_train)

# Predict with meta-model
y_pred_1 = meta_model1.predict(x_meta_test)

mse_1 = root_mean_squared_error(y_test, y_pred_1)
print(f'Mean Squared Error of Stacking Regressor: {mse_1:.2f}')#58.48